# Exemple de notebook - utilisation de jupysql

Documentation: https://jupysql.ploomber.io/


In [2]:
# Chargement de la librarie

%load_ext sql
%sql duckdb:///../../../database/data.duckdb
%config SqlMagic.displaylimit = 10

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../../database/data.duckdb'

Lister les tables:


In [3]:
%sqlcmd tables

Name
cog_communes
edc_communes
edc_prelevements
edc_resultats
laposte_communes
mapping_categories


Lister les colonnes d'une table:


In [4]:
%sqlcmd columns -t edc_prelevements

name,type,nullable,default,autoincrement,comment
cddept,VARCHAR,True,None,False,None
cdreseau,VARCHAR,True,None,False,None
inseecommuneprinc,VARCHAR,True,None,False,None
nomcommuneprinc,VARCHAR,True,None,False,None
cdreseauamont,VARCHAR,True,None,False,None
nomreseauamont,VARCHAR,True,None,False,None
pourcentdebit,VARCHAR,True,None,False,None
referenceprel,VARCHAR,True,None,False,None
dateprel,DATE,True,None,False,None
heureprel,VARCHAR,True,None,False,None


Exécuter une commande SQL sur une ligne:


In [5]:
%sql select * from mapping_categories

Running query in 'duckdb:///../../../database/data.duckdb'

cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
PESTOT,6276,TOTAL DES PESTICIDES ANALYSÉS,Total des pesticides analysés,None,pesticides
ATRZ,1107,ATRAZINE,Atrazine,1912-24-9,pesticides
SMZ,1263,SIMAZINE,Simazine,122-34-9,pesticides
MTC,1221,MÉTOLACHLORE,Métolachlore,51218-45-2,pesticides
TBZ,1268,TERBUTHYLAZIN,Terbuthylazin,5915-41-3,pesticides
METZCL,1670,MÉTAZACHLORE,Métazachlore,67129-08-02,pesticides
MTBZ,1225,MÉTRIBUZINE,Métribuzine,21087-64-9,pesticides
MTMI,1215,MÉTAMITRONE,Métamitrone,41394-05-02,pesticides
ALCL,1101,ALACHLORE,Alachlore,15972-60-8,pesticides
TERBU,1269,TERBUTRYNE,Terbutryne,886-50-0,pesticides


Exécuter une commande SQL sur plusieurs lignes


In [6]:
%%sql
with prelevements_unique as (
    select
        de_partition,
        referenceprel,
        inseecommuneprinc
    from edc_prelevements
    group by de_partition, referenceprel, inseecommuneprinc
)
select
    edc_resultats.de_partition , cdparametresiseeaux, cdparametre, libmajparametre, libminparametre, casparam, COUNT(*) as count, COUNT(DISTINCT inseecommuneprinc) as count_communes
from edc_resultats
left join prelevements_unique on
    edc_resultats.referenceprel = prelevements_unique.referenceprel
    and edc_resultats.de_partition = prelevements_unique.de_partition
GROUP BY edc_resultats.de_partition , cdparametresiseeaux, cdparametre, libmajparametre, libminparametre, casparam
ORDER BY edc_resultats.de_partition, count DESC;

Running query in 'duckdb:///../../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

de_partition,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,count,count_communes
2020,PH,1302,PH,pH,None,338110,34097
2020,CDT25,1303,CONDUCTIVITÉ À 25°C,Conductivité à 25°C,None,263505,34101
2020,TEAU,1301,TEMPÉRATURE DE L'EAU,Température de l'eau,None,262440,34027
2020,ECOLI,1449,ESCHERICHIA COLI /100ML - MF,Escherichia coli /100ml - MF,None,257230,34098
2020,STRF,6455,ENTÉROCOQUES /100ML-MS,Entérocoques /100ml-MS,None,257229,34097
2020,CTF,1447,BACTÉRIES COLIFORMES /100ML-MS,Bactéries coliformes /100ml-MS,None,257158,34098
2020,TURBNFU,1295,TURBIDITÉ NÉPHÉLOMÉTRIQUE NFU,Turbidité néphélométrique NFU,None,256959,34098
2020,ODQ,5901,ODEUR (QUALITATIF),Odeur (qualitatif),None,254069,33586
2020,GT36_44,5441,BACT. AÉR. REVIVIFIABLES À 36°-44H,Bact. aér. revivifiables à 36°-44h,None,253182,33647
2020,CL2LIB,1398,CHLORE LIBRE,Chlore libre,None,253105,33181
